## 6)Literacy group in India that has the highest percentage of people speaking three languages or more

In [1]:
#files used c19,c08(pop by literacy)

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

In [3]:
#ASSIGNING COL NAMES
column_names = ['State', 'District', 'Name', 'TRU','Edu-level','Persons.2','Males.2','Females.2','Persons.3','Males.3','Females.3']
#READING DATA
df = pd.read_excel('DDW-C19-0000.xlsx',names=column_names,skiprows=5,skipfooter=3)
#check skipfooter once again

In [4]:
df.columns

Index(['State', 'District', 'Name', 'TRU', 'Edu-level', 'Persons.2', 'Males.2',
       'Females.2', 'Persons.3', 'Males.3', 'Females.3'],
      dtype='object')

In [5]:
# df.head()

In [6]:
df.shape

(864, 11)

In [7]:
df['Edu-level'].unique()

array(['Total', 'Illiterate', 'Literate', 'Literate but below primary',
       'Primary but below middle', 'Middle but below matric/secondary',
       'Matric/Secondary but below graduate', 'Graduate and above'],
      dtype=object)

In [8]:
#TAKING ONLY REQUIRED COLUMNS
df = df[['State','Name','TRU','Edu-level','Persons.3']]
# REMOVING URBAN AND RURAL ROWS
# idx = df.loc[(df['TRU']!='Total') | (df['Edu-level'] == 'Total') | (df['Edu-level'] == 'Illiterate')|(df['Edu-level'] == 'Literate')].index
idx = df.loc[(df['TRU']!='Total') | (df['Edu-level'] == 'Total') |(df['Edu-level'] == 'Literate')].index
df = df.drop(idx)

In [9]:
df.reset_index(drop=True,inplace=True)

#### PREPROCESSING C08

In [10]:
column_names = ['table_name', 'State','District', 'Name', 'TRU','Age-group', 'Persons', 'Males', 'Females', 'Persons.1', 'Males.1', 'Females.1', 'Persons.2', 'Males.2', 'Females.2', 'Persons.3', 'Males.3', 'Females.3', 'Persons.4', 'Males.4', 'Females.4', 'Persons.5', 'Males.5', 'Females.5', 'Persons.6', 'Males.6', 'Females.6', 'Persons.7', 'Males.7', 'Females.7', 'Persons.8',
       'Males.8', 'Females.8', 'Persons.9', 'Males.9', 'Females.9', 'Persons.10', 'Males.10', 'Females.10', 'Persons.11', 'Males.11', 'Females.11', 'Persons.12', 'Males.12', 'Females.12']

In [11]:
# DDW-0000C-08.xlsx
lit_pop = pd.read_excel('DDW-0000C-08.xlsx',names=column_names,skiprows=6)

In [12]:
# lit_pop.head(10)

In [13]:
#MAKING SOME SPELLING CHANGES
lit_pop['Name'] = lit_pop['Name'].str.replace('State - ','')

In [14]:
lit_pop.shape

(3132, 45)

In [15]:
# REMOVING URBAN AND RURAL ROWS and AGE-GROUP "TOTAL" as they are not needed
idx = lit_pop.loc[(lit_pop['TRU']!='Total') | (lit_pop['Age-group'] !='All ages')].index
lit_pop = lit_pop.drop(idx)

In [16]:
lit_pop.reset_index(drop=True,inplace=True)

COMBINING BASED ON GROUPINGS WE NEED

In [17]:
newdf = pd.DataFrame()

In [18]:
# newdf['Name'] = lit_pop['Name']

newdf['ill'] = lit_pop['Persons.1']                             #ILLITERATE
newdf['lbp'] = lit_pop['Persons.3'] + lit_pop['Persons.4']   #LITERACY BUT BELOW PRIMARY
newdf['pbm'] = lit_pop['Persons.5']                         #PRIMARY BUT BELOW MIDDLE
newdf['mbm'] = lit_pop['Persons.6']                          # MIDDLE BUT BELOW MATRIC

newdf['mbg'] = lit_pop['Persons.7']+lit_pop['Persons.8']+lit_pop['Persons.9']+lit_pop['Persons.10']     #MATRIC BUT BELOW GRAD
newdf['gaa'] = lit_pop['Persons.11']                          #GRADUATE AND ABOVE


In [19]:
# TRANSPOSING MAKES each category values AS A ROW
# and then we convert rows into columns to ease calculations
# this will result in the format required for c19 file respectively

conv_df = pd.melt(newdf.transpose())

In [20]:
df['group_perc'] = df['Persons.3']/conv_df['value']

In [21]:
df.head()

,State,Name,TRU,Edu-level,Persons.3,group_perc
0,0,INDIA,Total,Illiterate,3879858,0.008676
1,0,INDIA,Total,Literate but below primary,3733616,0.020509
2,0,INDIA,Total,Primary but below middle,8636296,0.046893
3,0,INDIA,Total,Middle but below matric/secondary,10743350,0.080232
4,0,INDIA,Total,Matric/Secondary but below graduate,33899219,0.176381


In [22]:
df.columns

Index(['State', 'Name', 'TRU', 'Edu-level', 'Persons.3', 'group_perc'], dtype='object')

In [23]:
#finding index of maxratio
indexes = []
for i in range(36):
    t = i*6                        #number of categories in a state
    indexes.append(df.iloc[t:t+6,-1].idxmax())   #finding index of maxratio

In [24]:
#MAXRATIO ROWS ARE STORED INTO A DATAFRAME
finaldf = df.iloc[indexes]

In [25]:
#SORTING DF ON STATEID
finaldf = finaldf.sort_values(['State'])
finaldf.reset_index(drop=True,inplace=True)

In [26]:
#TAKING ONLY REQUIRED COLUMNS AND RENAMING
finaldf = finaldf[['State','Name','Edu-level','group_perc']]

In [27]:
resultdf = finaldf[['State','Edu-level','group_perc']]

In [28]:
resultdf.rename(columns={'State':'state/ut','Edu-level':'literacy-group','group_perc':'percentage'},inplace=True)
# resultdf.head()

In [29]:
#WRITING TO CSV
resultdf.to_csv('literacy-india.csv',index=False)

In [30]:
#completed